# Llama-Index Setup

In [2]:
import os
import pickle

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.query_engine import CitationQueryEngine
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core import Settings
from dotenv import load_dotenv
from llama_parse import LlamaParse
import nest_asyncio

embed_model = OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-3.5-turbo-0125")

Settings.llm = llm
Settings.embed_model = embed_model

# Read Data

In [3]:
load_dotenv()
nest_asyncio.apply()

In [23]:
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")

# Do not parse twice. Check if the file is already parsed
documents = None
if not os.path.exists("./parsed-objects/documents.pickle"):
    print("Parsing documents")
    parser = LlamaParse(
        api_key=LLAMA_CLOUD_API_KEY,
        result_type="text",
        verbose=True,
    )

    file_extractor = {".pdf": parser}
    documents = SimpleDirectoryReader(
        "./data", file_extractor=file_extractor
    ).load_data()
    with open("./parsed-objects/documents.pickle", "wb") as f:
        pickle.dump(documents, f)
else:
    print("Loading parsed documents from pickle")
    with open("./parsed-objects/documents.pickle", "rb") as f:
        documents = pickle.load(f)

assert documents is not None, "Documents are not loaded"

Parsing documents
Started parsing the file under job_id bc717ac5-542b-4e06-baf9-41e363548a0e
Started parsing the file under job_id 094d1094-fc6b-46e9-9047-df612c9793d1
Started parsing the file under job_id 16bee0f8-c980-438d-8941-7be1d5bc7965
Started parsing the file under job_id 16ed8372-31ca-49f7-b9b3-5633787ed16e
Started parsing the file under job_id ef8bf9c6-e69f-4029-8a3f-ae1f09d65d0a
Started parsing the file under job_id 5de8d21a-9f32-4003-8a35-ec7b06c5e6a3
Started parsing the file under job_id 8c51f3b3-9590-4424-a6c8-b04198220cf0
Started parsing the file under job_id 33730a5c-7505-46c5-8d8f-933d96f392f1
Started parsing the file under job_id 92fce70a-8975-4a61-bb81-9d46d326a4f0
Started parsing the file under job_id 09d3e2ba-54da-4b80-a208-8287cb3f956a
Started parsing the file under job_id 7acc4c6c-e41f-4da6-9f14-3d72a71eadf3


In [4]:
if not os.path.exists("./citation"):
    print("Building index")
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir="./citation")
else:
    print("Loading index from storage")
    index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./citation")
    )

Loading index from storage


In [5]:
# query_engine = CitationQueryEngine.from_args(
#     index,
#     similarity_top_k=10,
#     citation_chunk_size=1024,
# )
retriever = index.as_retriever()

In [6]:
nodes = retriever.retrieve("Telephon companies have been sued.")

In [9]:
nodes[0].text

'Case 3:07-cv-01697-MMC  Document 16  Filed 08/31/07    Page 1 of 2\n2                     E                                 #IR\n---\n        Case 3:07-cv-01697-MMC               Document 16      Filed 08/31/07       Page 2 of 2\n775t#\n         November 2, 2007                         10:30 a.m.             The parties shall file a joint Case\n       Management Statement no later than October 26, 2007.\n                August 31, 2007'

In [11]:
nodes[0].text

'Case 3:07-cv-01697-MMC  Document 16  Filed 08/31/07    Page 1 of 2\n2                     E                                 #IR\n---\n        Case 3:07-cv-01697-MMC               Document 16      Filed 08/31/07       Page 2 of 2\n775t#\n         November 2, 2007                         10:30 a.m.             The parties shall file a joint Case\n       Management Statement no later than October 26, 2007.\n                August 31, 2007'

In [72]:
print(nodes[1].text)

Case 3:07-cv-01697-ARC          Document 82      Filed 09/17/08     Page 1 of 4
                         IN THE UNITED STATES DISTRICT COURT
                     FOR THE MIDDLE DISTRICT OF PENNSYLVANIA
 DEBRA ANN WOODRUFF,                              CIVIL ACTION NO. 3:07-CV-1697
 Administratrix of the Estate of Jonathan
 Woodruff, and DEBRA ANN
 WOODRUFF, individually
         Plaintiff,
                v.                                (JUDGE CAPUTO)
 SULLIVAN COUNTY RURAL ELECTRIC
 COOPERATIVE, INC.,
 COMMONWEALTH TELEPHONE CO.,
 COMMONWEALTH TELEPHONE
 ENTERPRISES, EPIX INTERNET
 SERVICES, COMMONWEALTH
 COMMUNICATIONS and HENKELS &
 McCOY, INC.,
         Defendants.                MEMORANDUM ORDER
        Plaintiff Debra Ann Woodruff commenced this action by filing a complaint on
September 17, 2007.        The Complaint contains claims for common law negligence and
wrongful death pursuant to 42 Pa. Cons. Stat. Ann. § 8301 and Pa. R. Civ. P. No 2202(a)
and a Survival Action pursuan

# Document Summary Index

In [75]:
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

# LLM (gpt-3.5-turbo)
chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
splitter = SentenceSplitter(chunk_size=1024)

In [76]:
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    llm=chatgpt,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

/Users/dimatimofeev/Projects/stanford-law-hackathon-2024/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Summarizing documents:   0%|          | 0/11 [00:00<?, ?it/s]

current doc id: e4723477-04ff-4d35-b8b1-aa5dd7535857


Summarizing documents:   9%|▉         | 1/11 [00:04<00:44,  4.47s/it]

current doc id: c2ca99b1-6d12-40aa-9a36-b9cece67ffee


Summarizing documents:  18%|█▊        | 2/11 [00:06<00:29,  3.27s/it]

...

Summarizing documents:  82%|████████▏ | 9/11 [00:26<00:05,  2.83s/it]

current doc id: 048a642c-807c-41e2-adea-4ec439314eeb


Summarizing documents:  91%|█████████ | 10/11 [00:28<00:02,  2.73s/it]

current doc id: 4997c983-59b1-41f8-a45f-0601e68260aa


Generating embeddings: 100%|██████████| 11/11 [00:04<00:00,  2.64it/s]


In [77]:
# doc_summary_index.get_document_summary("Boston")

In [79]:
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # similarity_top_k=1,
)

In [88]:
retrieved_nodes = retriever.retrieve("Who has been sued in California?")

In [95]:
node = retrieved_nodes[0]

In [101]:
node.metadata["file_name"]

'gov.uscourts.cand.190473.14.0.pdf'